<a href="https://colab.research.google.com/github/eduardoriverosq/sintomatumorcerebral/blob/main/Atenci%C3%B3n_de_M%C3%BAltiples_Cabezas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atención de múltiples cabezas**


In [2]:
#Instalamos las librerías necesarias
!pip install transformers
!pip install bertviz
!pip install torch
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.1 MB/s eta 0:00:00


In [3]:
#Importamos las librerías
from transformers import BertModel, BertTokenizer
from bertviz import head_view
import torch
import pandas as pd


In [4]:
#Cargamos un vanilla Bert-base model.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [46]:
texto = "I have severe headaches and see blurred tumor"

tokens = tokenizer.encode(texto)
inputs = torch.tensor(tokens).unsqueeze(0)  #unsqueeze cambia la forma de (20, ) -> (1, 20)
print (inputs)


tensor([[  101,  1045,  2031,  5729, 14978,  2015,  1998,  2156, 18449, 13656,
           102]])


In [47]:
attention = model(inputs, output_attentions=True) [2] #Recoge las puntuaciones de atención de BERT

In [48]:
#average attention in the last encoder
final_attention = attention[-1].mean(1)[0]

In [49]:
attention_df = pd.DataFrame(final_attention.detach()).applymap(float).round(3)

attention_df.columns = tokenizer.convert_ids_to_tokens(tokens)
attention_df.index = tokenizer.convert_ids_to_tokens(tokens)

attention_df #sums across rows add up to 1. sums across columns do not

,[CLS],i,have,severe,headache,##s,and,see,blurred,tumor,[SEP]
[CLS],0.086,0.184,0.080,0.060,0.033,0.026,0.061,0.060,0.037,0.120,0.252
i,0.046,0.078,0.054,0.037,0.022,0.027,0.050,0.046,0.019,0.027,0.593
have,0.044,0.041,0.082,0.036,0.030,0.027,0.034,0.043,0.015,0.020,0.628
severe,0.032,0.035,0.043,0.064,0.034,0.027,0.028,0.038,0.019,0.017,0.663
headache,0.036,0.020,0.025,0.028,0.091,0.026,0.029,0.018,0.017,0.025,0.685
##s,0.038,0.018,0.024,0.029,0.040,0.088,0.040,0.022,0.018,0.017,0.666
and,0.054,0.027,0.026,0.026,0.021,0.030,0.133,0.028,0.022,0.022,0.612
see,0.038,0.036,0.047,0.030,0.020,0.023,0.037,0.085,0.025,0.027,0.630
blurred,0.039,0.029,0.026,0.026,0.021,0.018,0.026,0.032,0.069,0.045,0.670
tumor,0.066,0.038,0.029,0.023,0.036,0.019,0.028,0.036,0.056,0.102,0.566


In [50]:
#https://www-nlp.stanford.edu/pubs/clark2019what.pdf
#Capa índice 2 parece estar atendiendo al token previo
#Capa índice 6 parece ser para pronombres

In [51]:
tokens_as_list = tokenizer.convert_ids_to_tokens(inputs[0])
head_view(attention, tokens_as_list)

<IPython.core.display.Javascript object>

In [52]:
#Cabeza 3-1 atiende a tokens previos
head_view(attention, tokenizer.convert_ids_to_tokens(inputs[0]), layer=2, heads =[0])

<IPython.core.display.Javascript object>

In [53]:
#Head 8-10 relaciona objetos directos a sus verbos por ejemplo me -> dijo
head_view(attention, tokenizer.convert_ids_to_tokens(inputs[0]), layer=7, heads=[9])

<IPython.core.display.Javascript object>

In [54]:
#Atención en la décima cabeza del octavo codificador para ver la atención directa del objeto
eight_ten = attention[7] [0] [9]

In [55]:
#Obtener la matriz de atención
attention_df = pd.DataFrame(eight_ten.detach()).applymap(float).round(3)

attention_df.columns = tokenizer.convert_ids_to_tokens(tokens)
attention_df.index = tokenizer.convert_ids_to_tokens(tokens)

attention_df #Las sumas entre filas suman 1. Las sumas entre columnas no

,[CLS],i,have,severe,headache,##s,and,see,blurred,tumor,[SEP]
[CLS],0.003,0.005,0.002,0.002,0.002,0.004,0.001,0.002,0.008,0.012,0.960
i,0.026,0.016,0.002,0.002,0.007,0.003,0.001,0.001,0.003,0.016,0.923
have,0.079,0.160,0.033,0.003,0.005,0.010,0.012,0.001,0.001,0.009,0.686
severe,0.119,0.104,0.264,0.031,0.009,0.029,0.018,0.009,0.001,0.016,0.401
headache,0.042,0.043,0.101,0.020,0.007,0.015,0.016,0.005,0.001,0.003,0.747
##s,0.058,0.063,0.726,0.043,0.006,0.013,0.027,0.019,0.000,0.005,0.039
and,0.066,0.081,0.321,0.032,0.016,0.037,0.053,0.026,0.001,0.015,0.353
see,0.071,0.122,0.064,0.014,0.007,0.019,0.292,0.029,0.004,0.015,0.364
blurred,0.041,0.028,0.016,0.003,0.002,0.015,0.332,0.208,0.016,0.020,0.319
tumor,0.030,0.055,0.025,0.006,0.007,0.020,0.154,0.182,0.131,0.027,0.363
